In [1]:
import onnxruntime as ort
import glob as gb
import pandas as pd
import numpy as np

# Path to trained model
onnx_path = "./../neural_network/saved_models/NN_model_for_uq_analysis.onnx"

# Create session
session = ort.InferenceSession(
    onnx_path,
    providers=["CPUExecutionProvider"]
)

print("Input shape:", session.get_inputs()[0].shape)
print("Output shape:", session.get_outputs()[0].shape)


Input shape: ['N', 2]
Output shape: ['N', 7]


In [2]:
# Read the data from the FlameMaster files
fname = './../neural_network/data/chemtable_FVV_2D_Enthalpy/*.kg'
files = gb.glob(fname)
nfiles = len(files)


# Read column names from the second line of the first file
with open(files[0], 'r') as f:
    lines = f.readlines()
    column_names = lines[1].strip().split('\t')  # Read second line (index 1) and split by whitespace

# Create empty lists to store the data
data_flameMaster = []

# Load and concatenate the data into DataFrames
for f in files:

    df_flameMaster_temp = pd.DataFrame(np.loadtxt(f, skiprows=2, dtype=np.float64),columns=column_names)
    data_flameMaster.append(df_flameMaster_temp)

# Concatenate all data into final DataFrames
df_flameMaster_all = pd.concat(data_flameMaster, ignore_index=True)

#Computing diffusivity 
df_flameMaster_all['Diff [kg/ms]'] = df_flameMaster_all['lambda [W/mK]'] / df_flameMaster_all['cp [J/kgK]']

# Select the relevant columns for input and output
input_data = ['ProgVar', 'TotalEnthalpy [J/kg]']
referenceEnthalpy = 276240

# Shift the enthalpy values to be relative to the reference enthalpy
df_flameMaster_all['TotalEnthalpy [J/kg]'] = df_flameMaster_all['TotalEnthalpy [J/kg]'] - referenceEnthalpy

# Select the relevant columns for input and output
output_data = ['ProdRateProgVar [kg/m^3s]', 'temperature [K]', 'Y-CO', 'density', 'mu [kg/ms]', 'cp [J/kgK]', 'Diff [kg/ms]']

# Create DataFrames for input and output data
X_all = df_flameMaster_all[input_data].to_numpy()
Z_all = df_flameMaster_all[output_data].to_numpy()

# Load mask from trained model
mask = np.load("./../neural_network/train_test_mask.npy")

X_train = X_all[mask].astype(np.float32)
Z_train = Z_all[mask].astype(np.float32)
X_test  = X_all[~mask].astype(np.float32)
Z_test  = Z_all[~mask].astype(np.float32)


In [16]:
import numpy as np
import math
import time

# Choose polynomial order (start small)
order = 20  # try 3 or 4 first

# Standardize inputs using training statistics
x_mean = X_train.mean(axis=0)
x_std  = X_train.std(axis=0) + 1e-12

def standardize(X):
    return (X - x_mean) / x_std

# Pick a subset of training data for PCE fit
rng = np.random.default_rng(0)
k=5
P = (order + 1) * (order + 2) // 2
N_fit = k * P  

idx_fit = rng.choice(len(X_train), size=N_fit, replace=False)

X_fit = X_train[idx_fit].astype(np.float64)
Z_fit = Z_train[idx_fit].astype(np.float64)

X_fit_s = standardize(X_fit)
X_test_s = standardize(X_test.astype(np.float64))

print("X_fit_s shape:", X_fit_s.shape)
print("Z_fit shape:", Z_fit.shape)


X_fit_s shape: (1155, 2)
Z_fit shape: (1155, 7)


In [17]:
def hermite_polynomial(n, x):
    if n == 0:
        return np.ones_like(x)
    elif n == 1:
        return x
    return x * hermite_polynomial(n-1, x) - (n-1) * hermite_polynomial(n-2, x)

def orthonormal_hermite(n, x):
    return hermite_polynomial(n, x) / np.sqrt(math.factorial(n))

def total_order_basis_2d(p):
    # list of (i,j) with i+j <= p
    return [(i, j) for i in range(p+1) for j in range(p+1) if i + j <= p]

def design_matrix_2d_hermite(Xs, basis):
    """
    Xs: standardized inputs (N,2)
    returns A: (N, P) where P = len(basis)
    """
    z1 = Xs[:, 0]
    z2 = Xs[:, 1]
    N = Xs.shape[0]
    P = len(basis)
    A = np.zeros((N, P), dtype=np.float64)

    # precompute H_n(z1), H_n(z2) up to max degree
    max_deg = max(max(i, j) for i, j in basis)
    H1 = np.stack([orthonormal_hermite(n, z1) for n in range(max_deg+1)], axis=1)  # (N, max_deg+1)
    H2 = np.stack([orthonormal_hermite(n, z2) for n in range(max_deg+1)], axis=1)

    for k, (i, j) in enumerate(basis):
        A[:, k] = H1[:, i] * H2[:, j]
    return A

basis = total_order_basis_2d(order)
print("order:", order, "| P terms:", len(basis))

A_fit = design_matrix_2d_hermite(X_fit_s, basis)
A_test = design_matrix_2d_hermite(X_test_s, basis)

print("A_fit shape:", A_fit.shape)
print("A_test shape:", A_test.shape)


order: 20 | P terms: 231
A_fit shape: (1155, 231)
A_test shape: (16265, 231)


In [18]:
def fit_pce(A, Y, ridge=0.0):
    """
    Fit coefficients for multi-output regression:
      A: (N,P)
      Y: (N,7)
    Returns W: (P,7)
    """
    if ridge <= 0.0:
        # least squares
        W, *_ = np.linalg.lstsq(A, Y, rcond=None)
        return W
    else:
        # ridge: (A^T A + λI)W = A^T Y
        P = A.shape[1]
        ATA = A.T @ A
        ATY = A.T @ Y
        W = np.linalg.solve(ATA + ridge*np.eye(P), ATY)
        return W

t0 = time.time()
W = fit_pce(A_fit, Z_fit, ridge=1e-10)  # tiny ridge for stability
t1 = time.time()

Z_pred_test = A_test @ W  # (N_test,7)

print("Fit time (s):", t1 - t0)
print("Z_pred_test shape:", Z_pred_test.shape)


Fit time (s): 0.0034279823303222656
Z_pred_test shape: (16265, 7)


In [19]:
Z_test_ref = Z_test.astype(np.float64)

rmse = np.sqrt(np.mean((Z_pred_test - Z_test_ref)**2, axis=0))
# Relative RMSE normalized by std of reference (fair across scales)
std_ref = Z_test_ref.std(axis=0) + 1e-12
rrmse = rmse / std_ref

print("RMSE per output:", rmse)
print("rRMSE per output:", rrmse)
print("Mean rRMSE:", rrmse.mean())


RMSE per output: [5.62404393e+01 5.83886652e-01 1.30280325e-04 4.57544037e-04
 1.23450785e-08 3.77045428e-01 1.17262447e-08]
rRMSE per output: [0.09379182 0.00142986 0.02790794 0.00065319 0.00111228 0.00490079
 0.00083831]
Mean rRMSE: 0.01866202708164957


In [20]:
print(Z_pred_test[0,:])
print(Z_test_ref[0,:])

[-8.29640895e+00  7.19917778e+02  8.45891780e-06  1.90676752e+00
  3.39514337e-05  1.15997054e+03  4.40399146e-05]
[1.10153997e-10 7.20000977e+02 3.30841985e-08 1.90673995e+00
 3.39532999e-05 1.16000000e+03 4.40411204e-05]
